Contents:
* Tracking problem
* update
* predict
* update & predict code
* multivariate kalman filters

# Tracking

Problem:
<img src="L2/1.png" alt="Drawing" style="width: 200px;"/>

* using  Kalman filters [continuous, uni-modal]
* Using Gaussian (1-D):
$f(x) = \frac{1}{\sqrt{2\pi \sigma^2}}exp(\frac{-(x-\mu)^2}{\sigma^2})$
<img src="L2/2.png" alt="Drawing" style="width: 300px;"/>

In [8]:
import numpy as np

def gaussian(x, mu, sigma):
    y = 1/np.sqrt(2*np.pi*sigma**2)*np.exp((-0.5*(x-mu)**2)/sigma**2)
    return y

**Kalman filters: ** 
Measurement update vs Prediction. 

* The measurement update will use Bayes Rule - a product, multiplication. 
* The prediction update will use total probability - a convolution, an addition.

<img src="L2/3.png" alt="Drawing" style="width: 300px;"/>

<img src="L2/4.png" alt="Drawing" style="width: 300px;"/>

# Measurement update

<img src="L2/5.png" alt="Drawing" style="width: 300px;"/>

In [3]:
def update(mean1, var1, mean2, var2):
    new_mean = float((var2 * mean1 + var1 * mean2)) / (var1 + var2)
    new_var = 1 / (1/float(var1) + 1/float(var2))
    return [new_mean, new_var]

# Prediction update

<img src="L2/6.png" alt="Drawing" style="width: 300px;"/>

In [4]:
def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

# Wrap update and predict

In [7]:
# Kalman Filter Code
measurements = [5., 6., 7., 9., 10.] 
motion = [1., 1., 2., 1., 1.] 

measurement_sig = 4.
motion_sig = 2.

mu = 0.
sig = 10000.

for n in range(len(measurements)):
    [mu, sig] = update(mu, sig, measurements[n], measurement_sig)
    print 'update: ', [mu, sig]
    
    [mu, sig] = predict(mu, sig, motion[n], motion_sig)
    print 'predict: ', [mu, sig]

update:  [4.998000799680128, 3.9984006397441023]
predict:  [5.998000799680128, 5.998400639744102]
update:  [5.999200191953932, 2.399744061425258]
predict:  [6.999200191953932, 4.399744061425258]
update:  [6.999619127420922, 2.0951800575117594]
predict:  [8.999619127420921, 4.09518005751176]
update:  [8.999811802788143, 2.0235152416216957]
predict:  [9.999811802788143, 4.023515241621696]
update:  [9.999906177177365, 2.0058615808441944]
predict:  [10.999906177177365, 4.005861580844194]


# Multivariate Kalman filter

**Why in the multi-dimensional spaces**?????????????
In multi-dimensional spaces (like the real world, for example!) the Kalman filter not only allows you to estimate your positions, but also to infer your velocity from these position measurements. These inferred velocities then allow you to predict your future position.

<img src="L2/7.png" alt="Drawing" style="width: 300px;"/>
* correlation exits for the velocity and the position.

<img src="L2/8.png" alt="Drawing" style="width: 300px;"/>
* Further measurement (green lines) gives you high certain information about the velocity and the location.

Within Kalman filters there are two sub­sets:
1. Observables - momentary location
2. Hidden - in our example this was the
velocity. We could not measure it directly, only infer it from our position measurements.

When these two sub-sets interact, subsequent observations of the observables give us information about the hidden variables, so that we can make an estimate of what the hidden variable are.

# Kalman Filter Design

<img src="L2/9.png" alt="Drawing" style="width: 300px;"/>

<img src="L2/10.png" alt="Drawing" style="width: 400px;"/>
* This is just a generalization of the simple one-dimensional case. More details: https://www.udacity.com/wiki/cs373/kalman-filter-matrices

# Implement multivariate KF

In [46]:
from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print self.value[i]
        print ' '
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to add"
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to subtract"
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError, "Matrices must be m*n and n*p to multiply"
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise ValueError, "Matrix not positive-definite"
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


In [62]:
def kalman_filter(x, P):
    for n in range(len(measurements)):
        # measurement update
        Z = matrix([[measurements[n]]])
        y = Z * (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I * (K * H)) * P
        
        # prediction
        x = (F * x) + u
        P = F * P *F.transpose()
        
        return x, P

In [67]:
measurements = [1,2,3]
x = matrix([[0.], [0.]]) # initial state (location and velocity) 
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1], [0,1.]]) # next state function
H = matrix([[1., 0]]) #measurement function 
R = matrix([[1.]]) # measurement uncertainty 
I = matrix([[1., 0.], [0., 1.]]) # identity matrix

In [68]:
print kalman_filter(x, P)

([[0.0], [0.0]], [[999.0009990009988, 0.0], [0.0, 0.0]])


<img src="L2/11.png" alt="Drawing" style="width: 400px;"/>
* google cars use lasers and radar